In [ ]:
def generate_discrete_data(data,arrival_times,arrival_labels,last_arrival_time, extra_features = False, rand_off = False, subtract_baseline = True, WL_classifier = 100):

  coppied_arrival_times = np.copy(arrival_times)

  if rand_off == True:
    rand_offset_lst = np.round(np.random.normal(0,max_random_offset/3,len(arrival_times) - 2)).astype(int)
    for i in range(len(rand_offset_lst)):
      if rand_offset_lst[i] <= -max_random_offset or rand_offset_lst[i] >= max_random_offset:
        rand_offset_lst[i] = 0
    coppied_arrival_times[2:len(arrival_times)] += rand_offset_lst

  coppied_arrival_times = coppied_arrival_times.astype(int)

  N_pulses = 0
  for i in range(len(coppied_arrival_times)):
    if coppied_arrival_times[i] >= cont_starting_time and coppied_arrival_times[i] < last_arrival_time and arrival_labels[i] != -1:
      N_pulses += 1

  discrete_dataset = np.zeros((N_pulses,WL_classifier))
  discrete_labels = np.zeros(N_pulses)
  original_idx_lst = np.zeros(N_pulses)

  if extra_features == False:
    j = 0
    for i in range(len(coppied_arrival_times)):
      if coppied_arrival_times[i] >= cont_starting_time and coppied_arrival_times[i] < last_arrival_time and arrival_labels[i] != -1:
        if subtract_baseline == 'True':
          discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier] - data[coppied_arrival_times[i] - time_offset-1]
        else:
          discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier]
        discrete_labels[j] = arrival_labels[i]
        original_idx_lst[j] = i
        j += 1

    return discrete_dataset, discrete_labels, original_idx_lst

  else:

    classification_features = np.zeros((len(coppied_arrival_times),num_classification_features))
    attention_features = np.zeros((len(coppied_arrival_times),num_attention_features))
    classification_features_out = np.zeros((N_pulses,num_classification_features))
    attention_features_out = np.zeros((N_pulses,num_attention_features))


    for i in range(1,len(coppied_arrival_times) - 1):


      pulse = data[coppied_arrival_times[i] - new_starting_point:coppied_arrival_times[i] + pulse_len_after_starting] - data[coppied_arrival_times[i] - 1]
      pulse_bef = data[coppied_arrival_times[i-1] - new_starting_point:coppied_arrival_times[i-1] + pulse_len_after_starting] - data[coppied_arrival_times[i-1] - 1]
      pulse_aft = data[coppied_arrival_times[i+1] - new_starting_point:coppied_arrival_times[i+1] + pulse_len_after_starting] - data[coppied_arrival_times[i+1] - 1]
      delta_t_pulse_aft = coppied_arrival_times[i+1] - coppied_arrival_times[i]
      delta_t_pulse_bef = coppied_arrival_times[i] - coppied_arrival_times[i-1]

      t_rise2,amp2,qs,ql,psd_ratio,corr_n,corr_g = calc_specific_features_for_pulse(pulse,pulse_bef,pulse_aft, template_n,template_g, new_starting_point, pulse_len_after_starting,delta_t_pulse_aft)

      baseline = data[coppied_arrival_times[i] - 1]

      # to make sure we dont divide by 0.
      if qs == 0:
        qs = 1e-10
      if ql == 0:
        ql = 1e-10
      if amp2 == 0:
        amp2 = 1e-10
      if baseline == 0:
        baseline = 1e-10

      classification_features[i,:] = [t_rise2,amp2,qs,ql,psd_ratio,corr_n,corr_g,baseline]

    for i in range(2,len(coppied_arrival_times) - 2):

      delta_t_pulse_aft = coppied_arrival_times[i+1] - coppied_arrival_times[i]
      delta_t_pulse_bef = coppied_arrival_times[i] - coppied_arrival_times[i-1]
      baseline = classification_features[i,7]

      ql_ratio_before = classification_features[i,3]/classification_features[i-1,3]
      ql_baseline_ratio = classification_features[i,3]/baseline
      ql_ratio_after = classification_features[i,3]/classification_features[i+1,3]
      qs_ratio_before = classification_features[i,2]/classification_features[i-1,2]
      qs_ratio_after = classification_features[i,2]/classification_features[i+1,2]

      attention_features[i,:] = [delta_t_pulse_aft,delta_t_pulse_bef,baseline,ql_ratio_before,ql_baseline_ratio,ql_ratio_after,qs_ratio_before,
                                qs_ratio_after]

    j = 0
    for i in range(len(coppied_arrival_times)):
      if coppied_arrival_times[i] >= cont_starting_time and coppied_arrival_times[i] < last_arrival_time and arrival_labels[i] != -1:
        if subtract_baseline == 'True':
          discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier] - data[coppied_arrival_times[i] - time_offset-1]
        else:
          discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier]
        discrete_labels[j] = arrival_labels[i]
        original_idx_lst[j] = i
        classification_features_out[j,:] = classification_features[i,:]
        attention_features_out[j,:] = attention_features[i,:]
        j += 1

    return discrete_dataset, discrete_labels, original_idx_lst,classification_features_out,attention_features_out

def generate_discrete_data_with_FA(data,arrival_times,arrival_labels,last_arrival_time, extra_features = False, rand_off = False, subtract_baseline = True, WL_classifier = 100):

  coppied_arrival_times = np.copy(arrival_times)

  if rand_off == True:
    rand_offset_lst = np.round(np.random.normal(0,max_random_offset/3,len(arrival_times) - 2)).astype(int)
    for i in range(len(rand_offset_lst)):
      if rand_offset_lst[i] <= -max_random_offset or rand_offset_lst[i] >= max_random_offset:
        rand_offset_lst[i] = 0
    coppied_arrival_times[2:len(arrival_times)] += rand_offset_lst

  coppied_arrival_times = coppied_arrival_times.astype(int)

  N_pulses = 0
  for i in range(len(coppied_arrival_times)):
    if coppied_arrival_times[i] >= cont_starting_time and coppied_arrival_times[i] < last_arrival_time:
      N_pulses += 1

  discrete_dataset = np.zeros((N_pulses,WL_classifier))
  discrete_labels = np.zeros(N_pulses)
  original_idx_lst = np.zeros(N_pulses)

  if extra_features == False:
    j = 0
    for i in range(len(coppied_arrival_times)):
      if coppied_arrival_times[i] >= cont_starting_time and coppied_arrival_times[i] < last_arrival_time:
        if subtract_baseline == 'True':
          discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier] - data[coppied_arrival_times[i] - time_offset-1]
        else:
          discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier]
        discrete_labels[j] = arrival_labels[i]
        original_idx_lst[j] = i
        j += 1

    return discrete_dataset, discrete_labels, original_idx_lst

  else:

    classification_features = np.zeros((len(coppied_arrival_times),num_classification_features))
    attention_features = np.zeros((len(coppied_arrival_times),num_attention_features))
    classification_features_out = np.zeros((N_pulses,num_classification_features))
    attention_features_out = np.zeros((N_pulses,num_attention_features))


    for i in range(1,len(coppied_arrival_times) - 1):

      pulse = data[coppied_arrival_times[i] - new_starting_point:coppied_arrival_times[i] + pulse_len_after_starting] - data[coppied_arrival_times[i] - 1]
      pulse_bef = data[coppied_arrival_times[i-1] - new_starting_point:coppied_arrival_times[i-1] + pulse_len_after_starting] - data[coppied_arrival_times[i-1] - 1]
      pulse_aft = data[coppied_arrival_times[i+1] - new_starting_point:coppied_arrival_times[i+1] + pulse_len_after_starting] - data[coppied_arrival_times[i+1] - 1]
      delta_t_pulse_aft = coppied_arrival_times[i+1] - coppied_arrival_times[i]
      delta_t_pulse_bef = coppied_arrival_times[i] - coppied_arrival_times[i-1]

      t_rise2,amp2,qs,ql,psd_ratio,corr_n,corr_g = calc_specific_features_for_pulse(pulse,pulse_bef,pulse_aft, template_n,template_g, new_starting_point, pulse_len_after_starting,delta_t_pulse_aft)

      baseline = data[coppied_arrival_times[i] - 1]

      # to make sure we dont divide by 0.
      if qs == 0:
        qs = 1e-10
      if ql == 0:
        ql = 1e-10
      if amp2 == 0:
        amp2 = 1e-10
      if baseline == 0:
        baseline = 1e-10
      classification_features[i,:] = [t_rise2,amp2,qs,ql,psd_ratio,corr_n,corr_g,baseline]

    for i in range(2,len(coppied_arrival_times) - 2):

      delta_t_pulse_aft = coppied_arrival_times[i+1] - coppied_arrival_times[i]
      delta_t_pulse_bef = coppied_arrival_times[i] - coppied_arrival_times[i-1]
      baseline = classification_features[i,7]

      ql_ratio_before = classification_features[i,3]/classification_features[i-1,3]
      ql_baseline_ratio = classification_features[i,3]/baseline
      ql_ratio_after = classification_features[i,3]/classification_features[i+1,3]
      qs_ratio_before = classification_features[i,2]/classification_features[i-1,2]
      qs_ratio_after = classification_features[i,2]/classification_features[i+1,2]

      attention_features[i,:] = [delta_t_pulse_aft,delta_t_pulse_bef,baseline,ql_ratio_before,ql_baseline_ratio,ql_ratio_after,qs_ratio_before,
                                qs_ratio_after]

    j = 0
    for i in range(len(coppied_arrival_times)):
      if coppied_arrival_times[i] >= cont_starting_time and coppied_arrival_times[i] < last_arrival_time:
        if subtract_baseline == 'True':
          discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier] - data[coppied_arrival_times[i] - time_offset-1]
        else:
          discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier]
        discrete_labels[j] = arrival_labels[i]
        original_idx_lst[j] = i
        classification_features_out[j,:] = classification_features[i,:]
        attention_features_out[j,:] = attention_features[i,:]
        j += 1

    return discrete_dataset, discrete_labels, original_idx_lst,classification_features_out,attention_features_out



def generate_pulse_train(neutrons,gammas,lambda_n,lambda_g, return_statistics = False, add_noise = False):

  N_n, len_neutron = np.shape(neutrons)
  N_g, len_gamma = np.shape(gammas)
  max_length = max([len_neutron,len_gamma])
  n_exp_var_list = np.random.exponential(round(1/(Ts*lambda_n)), int(N_n/g_n_ratio))
  n_pulse_starting_points = np.array([int(round(sum(n_exp_var_list[:x]))) for x in range(1,int(N_n/g_n_ratio)+1)])
  g_exp_var_list = np.random.exponential(round(1/(Ts*lambda_g)), N_g)
  g_pulse_starting_points = np.array([int(round(sum(g_exp_var_list[:x]))) for x in range(1,N_g+1)])
  last_neutron_arriving_time = n_pulse_starting_points[len(n_pulse_starting_points)-1]
  last_gamma_arriving_time = g_pulse_starting_points[len(g_pulse_starting_points)-1]
  last_arrival_time = min(last_neutron_arriving_time,last_gamma_arriving_time)
  total_time = max(last_neutron_arriving_time,last_gamma_arriving_time) + 5*max_length + 1
  labels = np.zeros((total_time))

  for index in n_pulse_starting_points:
    while labels[index] != 0:
      index  += 1
    labels[index] = 1

  for index in g_pulse_starting_points:
    while labels[index] != 0:
      index  += 1
    labels[index] = 2

  if return_statistics == True:

    len_now = np.sum(labels != 0)
    true_inf = np.zeros((len_now,len_true_inf))
    neutron_features = generate_features_original_pulses(neutrons, template_n, template_g)
    gamma_features = generate_features_original_pulses(gammas, template_n, template_g)
    data = np.zeros((total_time))
    k,r,j = 0,0,0

    for i in range(len(labels)):
      if labels[i] == 1:
        data[i:i + max_length - starting_index] += neutrons[k,starting_index:]
        true_inf[j,0] = i # arrival location
        true_inf[j,1] = 0 # label
        true_inf[j,2] = k # serial number of neutron
        true_inf[j,3:3+num_original_features] = neutron_features[k,:]
        k += 1
        j += 1
      if labels[i] == 2:
        data[i:i + max_length - starting_index] += gammas[r,starting_index:]
        true_inf[j,0] = i
        true_inf[j,1] = 1
        true_inf[j,2] = r
        true_inf[j,3:3+num_original_features] = gamma_features[r,:]
        r += 1
        j += 1

    k,r,j = 0,0,0

    for i in range(len(labels)):
      if labels[i] == 1:
        if j != 0 and j != len_now - 1:
          true_inf[j,num_original_features + 3] = true_inf[j,0] - true_inf[j-1,0]
          true_inf[j,num_original_features + 4] = true_inf[j+1,0] - true_inf[j,0]
          true_inf[j,num_original_features + 5] = true_inf[j,11]/true_inf[j-1,11]
          true_inf[j,num_original_features + 6] = true_inf[j,11]/true_inf[j+1,11]
          true_inf[j,num_original_features + 7] = true_inf[j,12]/true_inf[j-1,12]
          true_inf[j,num_original_features + 8] = true_inf[j,12]/true_inf[j+1,12]

        k += 1
        j += 1
      if labels[i] == 2:
        if j != 0 and j != len_now - 1:
          true_inf[j,num_original_features + 3] = true_inf[j,0] - true_inf[j-1,0]
          true_inf[j,num_original_features + 4] = true_inf[j+1,0] - true_inf[j,0]
          true_inf[j,num_original_features + 5] = true_inf[j,11]/true_inf[j-1,11]
          true_inf[j,num_original_features + 6] = true_inf[j,11]/true_inf[j+1,11]
          true_inf[j,num_original_features + 7] = true_inf[j,12]/true_inf[j-1,12]
          true_inf[j,num_original_features + 8] = true_inf[j,12]/true_inf[j+1,12]
        r += 1
        j += 1

  else:
    len_now = np.sum(labels != 0)
    true_inf = np.zeros((len_now,2))
    data = np.zeros((total_time))
    k,r,j = 0,0,0

    for i in range(len(labels)):
      if labels[i] == 1:
        data[i:i + max_length - starting_index] += neutrons[k,starting_index:]
        true_inf[j,0] = i # arrival location
        true_inf[j,1] = 0 # label, 0 for neutrons
        k += 1
        j += 1
      if labels[i] == 2:
        data[i:i + max_length - starting_index] += gammas[r,starting_index:]
        true_inf[j,0] = i # arrival location
        true_inf[j,1] = 1 # label, 1 for gammas
        r += 1
        j += 1

  if add_noise == True:
    wgn = np.random.normal(noise_mean, std_added_noise, size=len(data))
    data = data + wgn

  return data,labels,true_inf,last_arrival_time

def generate_pulse_train_peak_labels(neutrons,gammas,lambda_n,lambda_g, add_noise = False):

  N_n, len_neutron = np.shape(neutrons)
  N_g, len_gamma = np.shape(gammas)
  max_length = max([len_neutron,len_gamma])
  n_exp_var_list = np.random.exponential(round(1/(Ts*lambda_n)), int(N_n/g_n_ratio))
  n_pulse_starting_points = np.array([int(round(sum(n_exp_var_list[:x]))) for x in range(1,int(N_n/g_n_ratio)+1)])
  g_exp_var_list = np.random.exponential(round(1/(Ts*lambda_g)), N_g)
  g_pulse_starting_points = np.array([int(round(sum(g_exp_var_list[:x]))) for x in range(1,N_g+1)])
  last_neutron_arriving_time = n_pulse_starting_points[int(N_n/g_n_ratio)-1]
  last_gamma_arriving_time = g_pulse_starting_points[N_g-1]
  last_arrival_time = min(last_neutron_arriving_time,last_gamma_arriving_time)
  total_time = max(last_neutron_arriving_time,last_gamma_arriving_time) + 3*max_length + 1
  labels = np.zeros((total_time))

  for index in n_pulse_starting_points:
    while labels[index] != 0:
      index  += 1
    labels[index] = 1

  for index in g_pulse_starting_points:
    while labels[index] != 0:
      index  += 1
    labels[index] = 2

  len_now = np.sum(labels != 0)
  true_inf = np.zeros((len_now,2))
  data = np.zeros((total_time))
  peak_labels = np.zeros(len(labels))
  true_inf_peaks = np.zeros((len_now,2))
  k,r,j = 0,0,0

  for i in range(len(labels)):
    if labels[i] == 1:
      data[i:i + max_length - starting_index] += neutrons[k,starting_index:]
      true_inf[j,0] = i # arrival location
      true_inf[j,1] = 0 # label
      peak_idx = int(np.argmax(neutrons[k,starting_index:starting_index + 200]) + starting_index)
      while(peak_labels[i + peak_idx] != 0):
        i += 1
      peak_labels[i + peak_idx] = 1
      true_inf_peaks[j,0] = i + peak_idx # arrival location
      true_inf_peaks[j,1] = 0 # label
      k += 1
      j += 1
    if labels[i] == 2:
      data[i:i + max_length - starting_index] += gammas[r,starting_index:]
      true_inf[j,0] = i
      true_inf[j,1] = 1
      peak_idx = int(np.argmax(gammas[r,starting_index:starting_index + 200]) + starting_index)
      while(peak_labels[i + peak_idx] != 0):
        i += 1
      peak_labels[i + peak_idx] = 1
      true_inf_peaks[j,0] = i + peak_idx # arrival location
      true_inf_peaks[j,1] = 1 # label
      r += 1
      j += 1

  if add_noise == True:
    wgn = np.random.normal(noise_mean, std_added_noise, size=len(data))
    data = data + wgn

  return data,labels,true_inf,peak_labels,true_inf_peaks,last_arrival_time

def correlation_coeff(x,y):
  sigma_x = np.std(x)
  sigma_y = np.std(y)
  mean_x = np.mean(x)
  mean_y = np.mean(y)
  N = len(x)
  sum = 0
  for i in range(N):
    sum += (x[i] - mean_x)*(y[i] - mean_y)
  sum = sum/(N*sigma_x*sigma_y)

  return sum

def generate_features_original_pulses(data, template_pulse_n, template_pulse_g):

  s1,s2 = np.shape(data)
  deriv_data = np.zeros((s1,L_deriv_orig_features))
  deriv_2_data = np.zeros((s1,L_deriv_orig_features))
  t_amp1 = np.zeros(s1,dtype=int) # amplitude index based on 2nd derivative
  t_rise1 = np.zeros(s1,dtype=int) # rise time based on first method amplitude
  t_amp2 = np.zeros(s1,dtype=int) # amplitude index based on maximum value
  t_rise2 = np.zeros(s1,dtype=int) # rise time based on second method amplitude
  qs = np.zeros(s1)
  ql = np.zeros(s1)
  psd_ratio = np.zeros(s1)
  amp1 = np.zeros(s1) # amplitude based on first method amplitude
  amp2 = np.zeros(s1) # amplitude based on second method amplitude
  tau1 = np.zeros(s1,dtype=int)# decay constant based on first method amplitude
  tau2 = np.zeros(s1,dtype=int) # decay constant based on second method amplitude
  corr_g = np.zeros(s1)
  corr_n = np.zeros(s1)

  count1 = 0
  for i in range(s1):
    for idx in range(L_deriv_orig_features):
      deriv_data[i,idx] = (data[i][idx+starting_index] - data[i][idx+starting_index-delta_orig_features])/delta_orig_features
      deriv_2_data[i,idx] = (deriv_data[i][idx] - ((data[i][idx+starting_index-delta_orig_features] - data[i][idx+starting_index-2*delta_orig_features])/delta_orig_features))/delta_orig_features

    flag1 = 0
    for idx in range(10,L_deriv_orig_features):
      if deriv_data[i,idx - 1] > 0 and deriv_data[i,idx] < 0 and deriv_2_data[i,idx] > 0:
        t_amp1[i] = idx + starting_index
        flag1 = 1
        break
    if flag1 == 0:
      count1 += 1

    amp1[i] = data[i,t_amp1[i]]
    low_tr = amp1[i]*0.1
    high_tr = amp1[i]*0.9

    flag1 = 0
    for idx in range(starting_index,starting_index + L_deriv_orig_features):
      if data[i,idx] > low_tr and flag1 == 0:
        time1 = idx
        flag1 = 1
      if data[i,idx] > high_tr:
        t_rise1[i] = idx - time1
        break

    t_amp2[i] = np.argmax(data[i,starting_index:starting_index + 200]) + starting_index

    amp2[i] = data[i,t_amp2[i]]
    low_tr = amp2[i]*0.1
    high_tr = amp2[i]*0.9

    flag1 = 0
    for idx in range(starting_index,L_deriv_orig_features+starting_index):
      if data[i,idx] > low_tr and flag1 == 0:
        time1 = idx
        flag1 = 1
      if data[i,idx] > high_tr:
        t_rise2[i] = idx - time1
        break

    for idx in range(t_amp1[i],s2):
      if data[i,idx] < amp1[i]*0.368:
        tau1[i] = idx - t_amp1[i]
        break

    for idx in range(t_amp2[i],s2):
      if data[i,idx] < amp2[i]*0.368:
        tau2[i] = idx - t_amp2[i]
        break

    qs[i] = np.sum(data[i,starting_index:starting_index+short_gate_orig_features])
    ql[i] = np.sum(data[i,starting_index:starting_index+long_gate_orig_features])
    psd_ratio[i] = (ql[i] - qs[i])/ql[i]

    corr_n[i] = correlation_coeff(data[i,starting_index:starting_index+template_length_orig_features],template_pulse_n[:template_length_orig_features])
    corr_g[i] = correlation_coeff(data[i,starting_index:starting_index+template_length_orig_features],template_pulse_g[:template_length_orig_features])

  output = np.concatenate((t_amp1[:,None],t_rise1[:,None],amp1[:,None],tau1[:,None],t_amp2[:,None],
                             t_rise2[:,None],amp2[:,None],tau2[:,None],qs[:,None],ql[:,None],psd_ratio[:,None],corr_n[:,None],corr_g[:,None]), axis=1)

  return output

def calc_specific_features_for_pulse(pulse,pulse_bef,pulse_aft, template_n, template_g, new_starting_point, pulse_len_after_starting,delta_t_pulse_aft):

  if delta_t_pulse_aft > 200:
    t_amp2 = np.argmax(pulse[new_starting_point:new_starting_point + 200]) + new_starting_point
  if delta_t_pulse_aft > 100 and delta_t_pulse_aft <= 200:
    t_amp2 = np.argmax(pulse[new_starting_point:new_starting_point + 100]) + new_starting_point
  if delta_t_pulse_aft <= 100:
    if delta_t_pulse_aft > 1:
      t_amp2 = np.argmax(pulse[new_starting_point:new_starting_point + delta_t_pulse_aft - 1]) + new_starting_point
    else:
      t_amp2 = 0

  amp2 = pulse[t_amp2]
  low_tr = amp2*0.1
  high_tr = amp2*0.9

  flag3 = 0
  flag1 = 0
  for idx in range(new_starting_point,pulse_len_after_starting+new_starting_point):
    time1 = 0
    if pulse[idx] > low_tr and flag1 == 0:
      time1 = idx
      flag1 = 1
    if pulse[idx] > high_tr:
      t_rise2 = idx - time1
      flag3 = 1
      break
  if flag3 == 0:
    t_rise2 = 0

  qs = np.sum(pulse[new_starting_point + start_gate_class_features:new_starting_point + start_gate_class_features+short_gate_class_features])
  ql = np.sum(pulse[new_starting_point + start_gate_class_features:new_starting_point + start_gate_class_features+long_gate_class_features])
  psd_ratio = (ql - qs)/ql

  corr_n = correlation_coeff(pulse[new_starting_point + time_offset + corr_start_class_features:new_starting_point + time_offset + corr_end_class_features],template_n[corr_start_class_features:corr_end_class_features])
  corr_g = correlation_coeff(pulse[new_starting_point + time_offset + corr_start_class_features:new_starting_point + time_offset + corr_end_class_features],template_g[corr_start_class_features:corr_end_class_features])


  return t_rise2,amp2,qs,ql,psd_ratio,corr_n,corr_g

def plot_data(data):
  pyplot.figure()
  pyplot.plot(np.arange(0,len(data)),data)
  pyplot.title('pulses')
  pyplot.xlabel('Time [ns]')
  pyplot.ylabel('Amplitude [mV]')
  pyplot.show()

def plot_learning_curve(losses, epoch_lst, train_accs, val_accs):
  """
  Plot the learning curve.
  """
  plt.title("Learning Curve: Loss per Iteration")
  plt.plot(epoch_lst, losses, label="Train")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.show()

  plt.title("Learning Curve: Accuracy per Iteration")
  plt.plot(epoch_lst, train_accs, label="Train")
  plt.plot(epoch_lst, val_accs, label="Validation")
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy")
  plt.legend(loc='best')
  plt.show()

def find_normal_parameters(data):
  mue = np.mean(data)
  sigma = np.std(data)
  return mue, sigma

def normalize_datasets(train,val,test,mue,sigma):
  norm_train = (train - mue)/sigma
  norm_val = (val - mue)/sigma
  norm_test = (test - mue)/sigma

  return norm_train,norm_val,norm_test

def create_arrivals_and_triggers(labels_in,labels_out,low,high,end):

  ## create arrays of the arrival times, trigger times, and corresponding labels, from the
  ## continuous time observations of both the true detector signal labels and the method triggers.
  non_zero_indices = np.where((labels_out[cont_starting_time + margin - low:end - margin + high] != 0))[0]
  triggers = non_zero_indices + cont_starting_time + margin - low
  non_zero_indices = np.where((labels_in[cont_starting_time + margin:end - margin] != 0))[0]
  arrivals = non_zero_indices + cont_starting_time + margin
  mask = (labels_in[low:high] != 0)
  class_labels = np.where(labels_in[low:high][mask] == 1, 1, 2)

  return arrivals,triggers,class_labels

def match_pairs(arrivals,triggers,low,high):

  d_range = int(low + high)
  ptr1 = 0
  ptr2 = 0
  triggered_events = -1*np.ones(len(arrivals))
  false_triggers_array = -1*np.ones(len(triggers))
  while(ptr1 < (len(arrivals) - 5) and ptr2 < (len(triggers) - 5)):
    if arrivals[ptr1 + 1] - arrivals[ptr1] > d_range:
      min_trig = arrivals[ptr1] - low
      max_trig = arrivals[ptr1] + high
      if triggers[ptr2] > max_trig:
        ptr1 += 1
        continue
      flag = 0
      ptr_new = ptr2
      curr_trigs = []

      while(triggers[ptr_new] <= max_trig):
        if triggers[ptr_new] < min_trig:
          ptr2 += 1
          ptr_new += 1
        else:
          curr_trigs.append(triggers[ptr_new])
          ptr_new += 1
      if curr_trigs == []:
        ptr1 += 1
        continue
      else:
        min_dist = abs(curr_trigs[0] - arrivals[ptr1])
        min_idx = 0
        for i in range(len(curr_trigs)):
          if abs(curr_trigs[i] - arrivals[ptr1]) < min_dist:
            min_dist = abs(curr_trigs[i] - arrivals[ptr1])
            min_idx = i
        triggered_events[ptr1] = triggers[ptr2 + min_idx]
        false_triggers_array[ptr2 + min_idx] = arrivals[ptr1]
        ptr2 = ptr_new
        ptr1 += 1

    else:
      flag = 0
      ptr_new1 = ptr1
      curr_arr = [arrivals[ptr_new1]]
      curr_trigs = []
      while(flag == 0):
        if arrivals[ptr_new1 + 1] - arrivals[ptr_new1] <= d_range:
          curr_arr.append(arrivals[ptr_new1 + 1])
          ptr_new1 += 1
        else:
          flag = 1

      min_trig = curr_arr[0] - low
      max_trig = curr_arr[len(curr_arr) - 1] + high
      if triggers[ptr2] > max_trig:
        ptr1 = ptr_new1 + 1
        continue
      else:
        ptr_new2 = ptr2
        while(triggers[ptr_new2] <= max_trig):
          if triggers[ptr_new2] < min_trig:
            ptr2 += 1
            ptr_new2 += 1
          else:
            curr_trigs.append(triggers[ptr_new2])
            ptr_new2 += 1
        out1, out_indexes = find_optimal_pairs(curr_arr, curr_trigs, -low, high)
        for i in range(np.shape(out_indexes)[1]):
          triggered_events[int(ptr1 + out_indexes[0,i])] = triggers[int(ptr2 + out_indexes[1,i])]
          false_triggers_array[int(ptr2 + out_indexes[1,i])] = arrivals[int(ptr1 + out_indexes[0,i])]
        ptr1 = ptr_new1 + 1
        ptr2 = ptr_new2

  return arrivals[:ptr1], triggers[:ptr2], triggered_events[:ptr1], false_triggers_array[:ptr2]

def pair_distance(pair):
    return pair[1] - pair[0]

def find_optimal_pairs(arr1, arr2, low, high):

    min_distance_sum = float('inf')
    optimal_pairs = []
    optimal_indexes = []
    min_len = min(len(arr1),len(arr2))
    for r in range(1,min_len + 1)[::-1]:
      all_perms_arr1 = []
      all_perms_arr1.extend(permutations(arr1, r))
      all_perms_arr2 = []
      all_perms_arr2.extend(permutations(arr2, r))
      for perm1 in all_perms_arr1:
          for perm2 in all_perms_arr2:
              pairs = list(zip(enumerate(perm1), enumerate(perm2)))
              if all(low <= pair_distance((pair[0][1], pair[1][1])) <= high for pair in pairs):
                  distance_sum = sum(abs(pair_distance((pair[0][1], pair[1][1]))) for pair in pairs)
                  if distance_sum < min_distance_sum:
                      min_distance_sum = distance_sum
                      optimal_pairs = [(pair[0][1], pair[1][1]) for pair in pairs]
                      optimal_indexes = [(pair[0][0], pair[1][0]) for pair in pairs]
      if optimal_pairs != []:
        break

    output_indexes = np.zeros((2,len(optimal_pairs)))
    for i in range(len(optimal_pairs)):
      opt1 = optimal_pairs[i][0]
      opt2 = optimal_pairs[i][1]
      for j in range(len(arr1)):
        if arr1[j] == opt1:
          arr1[j] = -1
          output_indexes[0,i] = j
          break
      for j in range(len(arr2)):
        if arr2[j] == opt2:
          arr2[j] = -1
          output_indexes[1,i] = j
          break

    return np.array(optimal_pairs), output_indexes

def calculate_statistics(triggered_events,false_triggers_array):
  all_counts = len(triggered_events) # Number of true events
  MD = np.count_nonzero(triggered_events == -1) # Number of miss-detections
  FA = np.count_nonzero(false_triggers_array == -1) # Number of false alarms
  tp = all_counts - MD # Number of true positives

  if tp + FA == 0:
    precision = 0
  else:
    precision = tp/(tp+FA)

  if tp + MD == 0:
    recall = 0
  else:
    recall = tp/(tp+MD)

  if precision + recall == 0:
    F1_score = 0
  else:
    F1_score = 2*precision*recall/(precision + recall)

  return all_counts,MD,FA,precision,recall,F1_score

def calc_count_information(true_labels,detected_labels,end_time,low,high):
  arrivals,triggers,class_labels = create_arrivals_and_triggers(true_labels,detected_labels,low,high,end_time)
  arrivals_out, triggers_out, triggered_events_out, false_triggers_array_out = match_pairs(arrivals,triggers,low,high)
  all_counts,MD,FA,precision,recall,F1_score = calculate_statistics(triggered_events_out,false_triggers_array_out)

  return all_counts,MD,FA,precision,recall,F1_score


def generate_template(pulses,template_length):

  s1,s2 = np.shape(pulses)
  template_pulse = np.zeros(template_length)
  for i in range(s1):
    template_pulse += pulses[i][starting_index:starting_index + template_length]
  template_pulse = template_pulse/max(template_pulse)

  return template_pulse

In [ ]:
def calc_deep_trigger_window_statistics(arrivals,triggers,trig_labels,true_labels):
  all_counts = len(arrivals)
  MD = np.count_nonzero(triggers == -1)
  FA = np.count_nonzero(trig_labels == -1)

  if np.count_nonzero(true_labels == 0) == 0:
    n_acc = 100
  else:
    n_acc = 100*(np.count_nonzero(trig_labels == 0)/np.count_nonzero(true_labels == 0))
  if np.count_nonzero(true_labels == 1) == 0:
    g_acc = 100
  else:
    g_acc = 100*(np.count_nonzero(trig_labels == 1)/np.count_nonzero(true_labels == 1))

  tp = all_counts - MD

  if tp + FA == 0:
    precision = 0
  else:
    precision = tp/(tp+FA)

  if tp + MD == 0:
    recall = 0
  else:
    recall = tp/(tp+MD)

  if precision + recall == 0:
    F1_score = 0
  else:
    F1_score = 2*precision*recall/(precision + recall)

  return all_counts,MD,FA,n_acc,g_acc,precision,recall,F1_score

def Deep_Trigger_statistics(N_windows,L_window,neutrons,gammas):

  statistics_mat = np.zeros((N_windows,8))

  iteration_num = 0
  while(True):
    perm = np.random.permutation(np.shape(neutrons)[0])
    neutrons = neutrons[perm]
    gammas = gammas[perm]
    noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                          lambda_g, return_statistics = False, add_noise = True)
    labels_NN = neural_networks_trigger(noised_data, last_arrival_time, prob=True)

    triggers_new_out = np.where(labels_NN != 0)[0]
    arrivals_out, triggers_out, triggered_events_out, false_triggers_array_out = match_pairs(true_inf[:,0],triggers_new_out,low_NN,high_NN)
    triggered_labels = np.where(false_triggers_array_out == -1, -1, labels[false_triggers_array_out.astype(int)] - 1)

    true_labels = labels[arrivals_out.astype(int)] - 1

    total_curr_length = arrivals_out[len(arrivals_out) - 1] - 2*cont_starting_time
    n_iter_curr = int(np.floor(total_curr_length/L_window))
    arrivals_idx_lst = [0]
    triggers_idx_lst = [0]
    curr_idx = 0
    for j in range(n_iter_curr):
      while(True):
        if arrivals_out[curr_idx] >= cont_starting_time + (j+1)*L_window:
          arrivals_idx_lst.append(curr_idx)
          break
        curr_idx += 1
    curr_idx = 0
    for j in range(n_iter_curr):
      while(True):
        if triggers_out[curr_idx] >= cont_starting_time + (j+1)*L_window:
          triggers_idx_lst.append(curr_idx)
          break
        curr_idx += 1
    for j in range(1,n_iter_curr):

      curr_arrivals = arrivals_out[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]
      curr_triggers = triggered_events_out[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]
      curr_triggered_labels = triggered_labels[triggers_idx_lst[j]:triggers_idx_lst[j + 1]]
      curr_true_labels = true_labels[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]
      curr_statistics = calc_deep_trigger_window_statistics(curr_arrivals,curr_triggers,curr_triggered_labels,curr_true_labels)
      statistics_mat[iteration_num,:] = curr_statistics
      iteration_num += 1

      if iteration_num == N_windows:
        return statistics_mat


def create_classifier_data_for_training_from_deep_trigger(neutrons,gammas,N_dataset,extra_feat_state):

  if extra_feat_state == True:

    discrete_data_out = np.zeros((N_dataset,WL_classifier))
    discrete_labels_out = np.zeros(N_dataset)
    classification_features_out = np.zeros((N_dataset,num_classification_features))
    attention_features_out = np.zeros((N_dataset,num_attention_features))
    N_mid = int(N_dataset/2)
    curr_n_count = 0
    curr_g_count = N_mid

    while(True):
      perm = np.random.permutation(np.shape(neutrons)[0])
      neutrons = neutrons[perm]
      gammas = gammas[perm]
      noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                            lambda_g, return_statistics = False, add_noise = True)

      deep_trig_arrival_times = deep_trigger_arrival_times(noised_data, last_arrival_time, prob=True)

      arrivals_out, triggers_out, triggered_events_out, false_triggers_array_out = match_pairs(true_inf[:,0],deep_trig_arrival_times,low_NN,high_NN)

      triggered_labels = np.where(false_triggers_array_out == -1, -1, labels[false_triggers_array_out.astype(int)] - 1)

      discrete_data, discrete_labels, original_idx_lst,classification_features,attention_features = generate_discrete_data(noised_data,triggers_out,triggered_labels,last_arrival_time, extra_features = extra_feat_state, rand_off = False, subtract_baseline = True, WL_classifier = WL_classifier)

      for i in range(len(discrete_labels)):
        if discrete_labels[i] == 0 and curr_n_count < N_mid:
          discrete_data_out[curr_n_count,:] = discrete_data[i,:]
          discrete_labels_out[curr_n_count] = discrete_labels[i]
          classification_features_out[curr_n_count,:] = classification_features[i,:]
          attention_features_out[curr_n_count,:] = attention_features[i,:]
          curr_n_count += 1
        if discrete_labels[i] == 1 and curr_g_count < N_dataset:
          discrete_data_out[curr_g_count,:] = discrete_data[i,:]
          discrete_labels_out[curr_g_count] = discrete_labels[i]
          classification_features_out[curr_g_count,:] = classification_features[i,:]
          attention_features_out[curr_g_count,:] = attention_features[i,:]
          curr_g_count += 1
        if curr_n_count == N_mid and curr_g_count == N_dataset:
          perm = np.random.permutation(np.shape(discrete_data_out)[0])
          discrete_data_out = discrete_data_out[perm]
          discrete_labels_out = discrete_labels_out[perm]
          classification_features_out = classification_features_out[perm]
          attention_features_out = attention_features_out[perm]

          return discrete_data_out, discrete_labels_out, classification_features_out, attention_features_out

  else:

    discrete_data_out = np.zeros((N_dataset,WL_classifier))
    discrete_labels_out = np.zeros(N_dataset)
    N_mid = int(N_dataset/2)
    curr_n_count = 0
    curr_g_count = N_mid

    while(True):
      perm = np.random.permutation(np.shape(neutrons)[0])
      neutrons = neutrons[perm]
      gammas = gammas[perm]
      noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                            lambda_g, return_statistics = False, add_noise = True)
      deep_trig_arrival_times = deep_trigger_arrival_times(noised_data, last_arrival_time, prob=True)

      arrivals_out, triggers_out, triggered_events_out, false_triggers_array_out = match_pairs(true_inf[:,0],deep_trig_arrival_times,low_NN,high_NN)

      triggered_labels = np.where(false_triggers_array_out == -1, -1, labels[false_triggers_array_out.astype(int)] - 1)

      discrete_data, discrete_labels, original_idx_lst = generate_discrete_data(noised_data,triggers_out,triggered_labels,last_arrival_time, extra_features = extra_feat_state, rand_off = False, subtract_baseline = True, WL_classifier = WL_classifier)

      for i in range(len(discrete_labels)):
        if discrete_labels[i] == 0 and curr_n_count < N_mid:
          discrete_data_out[curr_n_count,:] = discrete_data[i,:]
          discrete_labels_out[curr_n_count] = discrete_labels[i]
          curr_n_count += 1
        if discrete_labels[i] == 1 and curr_g_count < N_dataset:
          discrete_data_out[curr_g_count,:] = discrete_data[i,:]
          discrete_labels_out[curr_g_count] = discrete_labels[i]
          curr_g_count += 1
        if curr_n_count == N_mid and curr_g_count == N_dataset:
          perm = np.random.permutation(np.shape(discrete_data_out)[0])
          discrete_data_out = discrete_data_out[perm]
          discrete_labels_out = discrete_labels_out[perm]

          return discrete_data_out, discrete_labels_out

def create_classifier_data_for_training(neutrons,gammas,N_dataset,extra_feat_state):

  if extra_feat_state == True:

    discrete_data_out = np.zeros((N_dataset,WL_classifier))
    discrete_labels_out = np.zeros(N_dataset)
    classification_features_out = np.zeros((N_dataset,num_classification_features))
    attention_features_out = np.zeros((N_dataset,num_attention_features))
    N_mid = int(N_dataset/2)
    curr_n_count = 0
    curr_g_count = N_mid

    while(True):
      perm = np.random.permutation(np.shape(neutrons)[0])
      neutrons = neutrons[perm]
      gammas = gammas[perm]
      noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                            lambda_g, return_statistics = False, add_noise = True)

      discrete_data, discrete_labels, original_idx_lst,classification_features,attention_features = generate_discrete_data(noised_data,true_inf[:,0],true_inf[:,1],last_arrival_time, extra_features = extra_feat_state, rand_off = False, subtract_baseline = True, WL_classifier = WL_classifier)

      for i in range(len(discrete_labels)):
        if discrete_labels[i] == 0 and curr_n_count < N_mid:
          discrete_data_out[curr_n_count,:] = discrete_data[i,:]
          discrete_labels_out[curr_n_count] = discrete_labels[i]
          classification_features_out[curr_n_count,:] = classification_features[i,:]
          attention_features_out[curr_n_count,:] = attention_features[i,:]
          curr_n_count += 1
        if discrete_labels[i] == 1 and curr_g_count < N_dataset:
          discrete_data_out[curr_g_count,:] = discrete_data[i,:]
          discrete_labels_out[curr_g_count] = discrete_labels[i]
          classification_features_out[curr_g_count,:] = classification_features[i,:]
          attention_features_out[curr_g_count,:] = attention_features[i,:]
          curr_g_count += 1
        if curr_n_count == N_mid and curr_g_count == N_dataset:
          perm = np.random.permutation(np.shape(discrete_data_out)[0])
          discrete_data_out = discrete_data_out[perm]
          discrete_labels_out = discrete_labels_out[perm]
          classification_features_out = classification_features_out[perm]
          attention_features_out = attention_features_out[perm]

          return discrete_data_out, discrete_labels_out, classification_features_out, attention_features_out

  else:

    discrete_data_out = np.zeros((N_dataset,WL_classifier))
    discrete_labels_out = np.zeros(N_dataset)
    N_mid = int(N_dataset/2)
    curr_n_count = 0
    curr_g_count = N_mid

    while(True):
      perm = np.random.permutation(np.shape(neutrons)[0])
      neutrons = neutrons[perm]
      gammas = gammas[perm]
      noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                            lambda_g, return_statistics = False, add_noise = True)

      discrete_data, discrete_labels, original_idx_lst = generate_discrete_data(noised_data,true_inf[:,0],true_inf[:,1],last_arrival_time, extra_features = extra_feat_state, rand_off = False, subtract_baseline = True, WL_classifier = WL_classifier)

      for i in range(len(discrete_labels)):
        if discrete_labels[i] == 0 and curr_n_count < N_mid:
          discrete_data_out[curr_n_count,:] = discrete_data[i,:]
          discrete_labels_out[curr_n_count] = discrete_labels[i]
          curr_n_count += 1
        if discrete_labels[i] == 1 and curr_g_count < N_dataset:
          discrete_data_out[curr_g_count,:] = discrete_data[i,:]
          discrete_labels_out[curr_g_count] = discrete_labels[i]
          curr_g_count += 1
        if curr_n_count == N_mid and curr_g_count == N_dataset:
          perm = np.random.permutation(np.shape(discrete_data_out)[0])
          discrete_data_out = discrete_data_out[perm]
          discrete_labels_out = discrete_labels_out[perm]

          return discrete_data_out, discrete_labels_out


In [ ]:
def classifiers_statistics_with_deep_trigger(N_windows,L_window,neutrons,gammas):

  statistics_mat = np.zeros((N_windows,8))
  statistics_mat_CI = np.zeros((N_windows,4))
  statistics_mat_corr = np.zeros((N_windows,4))
  statistics_mat_FC_NN = np.zeros((N_windows,4))
  statistics_mat_concat = np.zeros((N_windows,4))
  statistics_mat_attention = np.zeros((N_windows,4))

  iteration_num = 0
  while(True):
    perm = np.random.permutation(np.shape(neutrons)[0])
    neutrons = neutrons[perm]
    gammas = gammas[perm]
    noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                          lambda_g, return_statistics = False, add_noise = True)


    labels_NN = neural_networks_trigger(noised_data, last_arrival_time, prob=True)

    triggers_new_out = []
    for s in range(len(labels_NN)):
      if labels_NN[s] != 0:
        triggers_new_out.append(s)

    arrivals_out, triggers_out, triggered_events_out, false_triggers_array_out = match_pairs(true_inf[:,0],triggers_new_out,low_NN,high_NN)

    triggered_labels = np.zeros(len(triggers_out))
    for s in range(len(triggers_out)):
      if false_triggers_array_out[s] == -1:
        triggered_labels[s] = -1
      else:
        triggered_labels[s] = labels[int(false_triggers_array_out[s])] - 1

    true_labels = np.zeros(len(arrivals_out))
    for s in range(len(arrivals_out)):
      true_labels[s] = labels[int(arrivals_out[s])] - 1


    discrete_data, discrete_labels, original_idx_lst,classification_features,attention_features = generate_discrete_data_with_FA(noised_data,triggers_out,triggered_labels,last_arrival_time, extra_features = True, rand_off = False, subtract_baseline = True, WL_classifier = WL_classifier)

    CI_labels, CI_values = charge_integration(discrete_data,CI_parameters)
    corr_labels, corr_values = correlation_classifier(discrete_data,Corr_parameters)
    FC_NN_labels, FC_NN_values = create_labels_Model_FC_NN(model_fc_nn,discrete_data,norm_params_model_fc_nn)
    concat_labels, concat_values = create_labels_Model_Concat(model_concat,discrete_data,classification_features,attention_features,norm_params_model_concat)
    attention_labels, attention_values = create_labels_Model_Attention(model_attention,discrete_data,classification_features,attention_features,norm_params_model_attention)

    total_curr_length = last_arrival_time - 2*cont_starting_time
    n_iter_curr = int(np.floor(total_curr_length/L_window))

    if triggers_out[0] < cont_starting_time:
      zero_idx = 0
      while(True):
        if triggers_out[zero_idx] >= cont_starting_time:
          break
        zero_idx += 1

      triggers_out = triggers_out[zero_idx:]
      triggered_labels = triggered_labels[zero_idx:]

    arrivals_idx_lst = [0]
    triggers_idx_lst = [0]
    curr_idx = 0
    for j in range(n_iter_curr):
      while(True):
        if arrivals_out[curr_idx] >= cont_starting_time + (j+1)*L_window:
          arrivals_idx_lst.append(curr_idx)
          break
        curr_idx += 1
    curr_idx = 0
    for j in range(n_iter_curr):
      while(True):
        if triggers_out[curr_idx] >= cont_starting_time + (j+1)*L_window:
          triggers_idx_lst.append(curr_idx)
          break
        curr_idx += 1


    for j in range(1,n_iter_curr):

      curr_arrivals = arrivals_out[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]
      curr_triggers = triggered_events_out[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]
      curr_triggered_labels = discrete_labels[triggers_idx_lst[j]:triggers_idx_lst[j + 1]]
      curr_true_labels = true_labels[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]

      statistics_mat[iteration_num,:] = calc_deep_trigger_window_statistics(curr_arrivals,curr_triggers,curr_triggered_labels,curr_true_labels)

      curr_CI_labels = CI_labels[triggers_idx_lst[j]:triggers_idx_lst[j + 1]]
      curr_corr_labels = corr_labels[triggers_idx_lst[j]:triggers_idx_lst[j + 1]]
      curr_FC_NN_labels = FC_NN_labels[triggers_idx_lst[j]:triggers_idx_lst[j + 1]]
      curr_concat_labels = concat_labels[triggers_idx_lst[j]:triggers_idx_lst[j + 1]]
      curr_attention_labels = attention_labels[triggers_idx_lst[j]:triggers_idx_lst[j + 1]]

      statistics_mat_CI[iteration_num,:] = calc_classifiers_and_deep_trigger_window_statistics(curr_CI_labels,curr_triggered_labels)
      statistics_mat_corr[iteration_num,:] = calc_classifiers_and_deep_trigger_window_statistics(curr_corr_labels,curr_triggered_labels)
      statistics_mat_FC_NN[iteration_num,:] = calc_classifiers_and_deep_trigger_window_statistics(curr_FC_NN_labels,curr_triggered_labels)
      statistics_mat_concat[iteration_num,:] = calc_classifiers_and_deep_trigger_window_statistics(curr_concat_labels,curr_triggered_labels)
      statistics_mat_attention[iteration_num,:] = calc_classifiers_and_deep_trigger_window_statistics(curr_attention_labels,curr_triggered_labels)

      iteration_num += 1

      if iteration_num == N_windows:
        return statistics_mat,statistics_mat_CI,statistics_mat_corr,statistics_mat_FC_NN,statistics_mat_concat,statistics_mat_attention


def calc_classifiers_and_deep_trigger_window_statistics(method_labels,true_labels):

  all_counts = np.count_nonzero(true_labels != -1)

  if all_counts == 0:
    acc = 100
  else:
    acc = 100*(np.count_nonzero(method_labels == true_labels)/all_counts)

  num_neutrons = np.count_nonzero(true_labels == 0)
  num_gammas = np.count_nonzero(true_labels == 1)

  num_correct_neutrons = np.count_nonzero(np.logical_and(true_labels == 0, true_labels == method_labels))
  num_correct_gammas = np.count_nonzero(np.logical_and(true_labels == 1, true_labels == method_labels))

  if num_neutrons == 0:
    neutron_acc = 100
  else:
    neutron_acc = 100*(num_correct_neutrons/num_neutrons)

  if num_gammas == 0:
    gamma_acc = 100
  else:
    gamma_acc = 100*(num_correct_gammas/num_gammas)

  return all_counts, acc, neutron_acc, gamma_acc

In [ ]:
def classifiers_statistics_known_arrivals(N_windows,L_window,neutrons,gammas):

  statistics_mat_CI = np.zeros((N_windows,4))
  statistics_mat_corr = np.zeros((N_windows,4))
  statistics_mat_FC_NN = np.zeros((N_windows,4))
  statistics_mat_concat = np.zeros((N_windows,4))
  statistics_mat_attention = np.zeros((N_windows,4))

  iteration_num = 0
  while(True):
    perm = np.random.permutation(np.shape(neutrons)[0])
    neutrons = neutrons[perm]
    gammas = gammas[perm]
    noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                          lambda_g, return_statistics = False, add_noise = True)

    discrete_data, discrete_labels, original_idx_lst,classification_features,attention_features = generate_discrete_data(noised_data,true_inf[:,0],true_inf[:,1],last_arrival_time, extra_features = True, rand_off = False, subtract_baseline = True, WL_classifier = WL_classifier)

    CI_labels, CI_values = charge_integration(discrete_data,CI_parameters)
    corr_labels, corr_values = correlation_classifier(discrete_data,Corr_parameters)
    FC_NN_labels, FC_NN_values = create_labels_Model_FC_NN(model_fc_nn,discrete_data,norm_params_model_fc_nn)
    concat_labels, concat_values = create_labels_Model_Concat(model_concat,discrete_data,classification_features,attention_features,norm_params_model_concat)
    attention_labels, attention_values = create_labels_Model_Attention(model_attention,discrete_data,classification_features,attention_features,norm_params_model_attention)

    now_arr_times = true_inf[:,0]

    total_curr_length = last_arrival_time - 2*cont_starting_time
    n_iter_curr = int(np.floor(total_curr_length/L_window))

    zero_idx = 0
    while(True):
      if now_arr_times[zero_idx] >= cont_starting_time:
        break
      zero_idx += 1

    updated_now_arr_times = now_arr_times[zero_idx:]
    arrivals_idx_lst = [0]
    curr_idx = 0
    for j in range(n_iter_curr):
      while(True):
        if updated_now_arr_times[curr_idx] >= cont_starting_time + (j+1)*L_window:
          arrivals_idx_lst.append(curr_idx)
          break
        curr_idx += 1

    for j in range(1,n_iter_curr):

          curr_CI_labels = CI_labels[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]
          curr_corr_labels = corr_labels[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]
          curr_FC_NN_labels = FC_NN_labels[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]
          curr_concat_labels = concat_labels[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]
          curr_attention_labels = attention_labels[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]
          curr_true_labels = discrete_labels[arrivals_idx_lst[j]:arrivals_idx_lst[j + 1]]

          statistics_mat_CI[iteration_num,:] = calc_classifiers_known_arrivals_window_statistics(curr_CI_labels,curr_true_labels)
          statistics_mat_corr[iteration_num,:] = calc_classifiers_known_arrivals_window_statistics(curr_corr_labels,curr_true_labels)
          statistics_mat_FC_NN[iteration_num,:] = calc_classifiers_known_arrivals_window_statistics(curr_FC_NN_labels,curr_true_labels)
          statistics_mat_concat[iteration_num,:] = calc_classifiers_known_arrivals_window_statistics(curr_concat_labels,curr_true_labels)
          statistics_mat_attention[iteration_num,:] = calc_classifiers_known_arrivals_window_statistics(curr_attention_labels,curr_true_labels)
          iteration_num += 1

          if iteration_num == N_windows:
            return statistics_mat_CI,statistics_mat_corr,statistics_mat_FC_NN,statistics_mat_concat,statistics_mat_attention

def calc_classifiers_known_arrivals_window_statistics(method_labels,true_labels):

  all_counts = len(true_labels)
  if all_counts == 0:
    acc = 100
  else:
    acc = 100*(np.count_nonzero(method_labels == true_labels)/all_counts)

  num_neutrons = np.count_nonzero(true_labels == 0)
  num_gammas = np.count_nonzero(true_labels == 1)

  num_correct_neutrons = np.count_nonzero(np.logical_and(true_labels == 0, true_labels == method_labels))
  num_correct_gammas = np.count_nonzero(np.logical_and(true_labels == 1, true_labels == method_labels))

  if num_neutrons == 0:
    neutron_acc = 100
  else:
    neutron_acc = 100*(num_correct_neutrons/num_neutrons)

  if num_gammas == 0:
    gamma_acc = 100
  else:
    gamma_acc = 100*(num_correct_gammas/num_gammas)

  return all_counts, acc, neutron_acc, gamma_acc

In [ ]:
def create_SDL_signal(data, SDL_delay, SDL_coeff):
    SDL_delay = int(SDL_delay)
    SDL_coeff = int(SDL_coeff)

    delayed_data = data[SDL_delay:]
    scaled_data = SDL_coeff * data[:len(data) - SDL_delay]

    if SDL_delay != 0:
      SDL_signal = np.concatenate((np.zeros(SDL_delay),delayed_data - scaled_data))
    else:
      SDL_signal = delayed_data - scaled_data

    return SDL_signal

def create_SDL_THR_labels(data,end,thr,jump_deriv):

  jump_deriv = int(jump_deriv)
  labels_out = np.zeros(len(data))
  i = cont_starting_time

  while(i  < end):
    if thr > data[i-1]  and data[i] > thr:
      labels_out[i] = 1
      i += jump_deriv
    i+=1

  return labels_out

def generate_correlation_data(data, template, temp_start, temp_end):
    temp_start = int(temp_start)
    temp_end = int(temp_end)
    current_template = template[temp_start:temp_end]
    sigma_y = np.std(current_template)
    mean_y = np.mean(current_template)
    N = len(current_template)
    corr_data_out = np.zeros(len(data))

    x_slices = np.lib.stride_tricks.sliding_window_view(data, window_shape=(temp_end - temp_start,))
    sigma_x = np.std(x_slices, axis=1)
    mean_x = np.mean(x_slices, axis=1)

    normalized_x = (x_slices - mean_x[:, np.newaxis]) / sigma_x[:, np.newaxis]
    normalized_template = (current_template - mean_y) / sigma_y

    corr_data = np.sum(normalized_x * normalized_template, axis=1) / N
    corr_data_out[:len(corr_data)] = corr_data

    return corr_data_out

def generate_correlation_data_old(data, template, temp_start, temp_end):
    temp_start = int(temp_start)
    temp_end = int(temp_end)
    current_template = template[temp_start:temp_end]
    sigma_y = np.std(current_template)
    mean_y = np.mean(current_template)
    N = len(current_template)
    corr_data_out = np.zeros(len(data))
    normalized_template = (current_template - mean_y) / sigma_y

    jumpy = 100000
    j = 0
    for i in range(0,len(data) - (temp_end - temp_start) - jumpy,jumpy):

      x_slices = np.lib.stride_tricks.sliding_window_view(data[i:i + jumpy + (temp_end - temp_start) + 5], window_shape=(temp_end - temp_start,))
      sigma_x = np.std(x_slices, axis=1)
      mean_x = np.mean(x_slices, axis=1)
      normalized_x = (x_slices - mean_x[:, np.newaxis]) / sigma_x[:, np.newaxis]
      corr_data = np.sum(normalized_x * normalized_template, axis=1) / N
      corr_data_out[i:i + len(corr_data)] = corr_data
      j = i + len(corr_data)

    if j < (len(data) - temp_end):
      for i in range(j,(len(data) - temp_end)):
        x = data[i + temp_start:i+temp_end]
        sigma_x = np.std(x)
        mean_x = np.mean(x)
        corr_data_out[i] = np.dot((x - mean_x), (current_template - mean_y)) / (N * sigma_x * sigma_y)

    return corr_data_out

def generate_correlation_data_old1(data, template, temp_start, temp_end):
    temp_start = int(temp_start)
    temp_end = int(temp_end)
    current_template = template[temp_start:temp_end]
    sigma_y = np.std(current_template)
    mean_y = np.mean(current_template)
    N = len(current_template)
    corr_data = np.zeros(len(data))

    for i in range(len(data) - temp_end):
      x = data[i + temp_start:i+temp_end]
      sigma_x = np.std(x)
      mean_x = np.mean(x)
      corr_data[i] = np.dot((x - mean_x), (current_template - mean_y)) / (N * sigma_x * sigma_y)

    return corr_data

def create_Corr_THR_labels(data1,data2,end,thr,jump_deriv):

  jump_deriv = int(jump_deriv)
  labels_out = np.zeros(len(data1))
  i = cont_starting_time

  while(i  < end):
    if (thr > data1[i-1]  and data1[i] > thr) or (thr > data2[i-1]  and data2[i] > thr):
      labels_out[i] = 1
      i += jump_deriv
    i+=1

  return labels_out

In [ ]:
def Deep_Trigger_Only_Results(result_trig_iters,neutrons,gammas):

  statistics_mat = np.zeros((3,8))

  for j in range(result_trig_iters):
    perm = np.random.permutation(np.shape(neutrons)[0])
    neutrons = neutrons[perm]
    gammas = gammas[perm]
    noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                          lambda_g, return_statistics = False, add_noise = True)

    deep_trig_arrival_times = deep_trigger_arrival_times(noised_data, last_arrival_time, prob=True)
    deep_trig_arrival_times_NM = deep_trigger_arrival_times_NM(noised_data, last_arrival_time, prob=True)
    corr_arr_times = corr_trig_arrival_times(noised_data,last_arrival_time,BM_corr_params)

    statistics_mat[0,:] += Deep_Trigger_statistics_results(deep_trig_arrival_times,labels,true_inf,low_NN,high_NN)
    statistics_mat[1,:] += Deep_Trigger_statistics_results(deep_trig_arrival_times_NM,labels,true_inf,low_NN,high_NN)
    statistics_mat[2,:] += Deep_Trigger_statistics_results(corr_arr_times,labels,true_inf,BM_corr_params[4],(low_NN + high_NN) - BM_corr_params[4])

  return statistics_mat/result_trig_iters

In [ ]:
def Deep_Trigger_All_BM_Results(result_trig_iters,neutrons,gammas):

  statistics_mat = np.zeros((7,8))

  for j in range(result_trig_iters):
    perm = np.random.permutation(np.shape(neutrons)[0])
    neutrons = neutrons[perm]
    gammas = gammas[perm]
    noised_data,labels,true_inf,cont_peak_labels,true_inf_peaks,last_arrival_time = generate_pulse_train_peak_labels(neutrons,gammas,lambda_n,
                                                                                          lambda_g, add_noise = True)

    deep_trig_arrival_times = deep_trigger_arrival_times(noised_data, last_arrival_time, prob=True)
    BM1_arr_times = BM1_arrival_times(noised_data,last_arrival_time,BM1_params)
    BM2_arr_times = BM2_arrival_times(noised_data,last_arrival_time,BM2_params)
    corr_n_trig_arr_times = corr_n_trig_arrival_times(noised_data,last_arrival_time,corr_n_trig_params)
    corr_g_trig_arr_times = corr_g_trig_arrival_times(noised_data,last_arrival_time,corr_g_trig_params)
    SDL_normal_THR_arr_times = SDL_normal_THR_arrival_times(noised_data,last_arrival_time,SDL_normal_THR_params)
    SDL_find_peaks_arr_times = SDL_find_peaks_arrival_times(noised_data,last_arrival_time,SDL_find_peaks_params)

    statistics_mat[0,:] += Deep_Trigger_statistics_results(deep_trig_arrival_times,labels,true_inf,low_NN,high_NN)
    statistics_mat[1,:] += Deep_Trigger_statistics_results(BM1_arr_times,labels,true_inf,BM1_params[5],(low_NN + high_NN) - BM2_params[5])
    statistics_mat[2,:] += Deep_Trigger_statistics_results(BM2_arr_times,labels,true_inf,BM2_params[2],(low_NN + high_NN) - BM2_params[2])
    statistics_mat[3,:] += Deep_Trigger_statistics_results(corr_n_trig_arr_times,labels,true_inf,corr_n_trig_params[4],(low_NN + high_NN) - corr_n_trig_params[4])
    statistics_mat[4,:] += Deep_Trigger_statistics_results(corr_g_trig_arr_times,labels,true_inf,corr_g_trig_params[4],(low_NN + high_NN) - corr_g_trig_params[4])
    statistics_mat[5,:] += Deep_Trigger_statistics_results(SDL_normal_THR_arr_times,labels,true_inf,SDL_normal_THR_params[4],(low_NN + high_NN) - SDL_normal_THR_params[4])
    statistics_mat[6,:] += Deep_Trigger_statistics_results(SDL_find_peaks_arr_times,cont_peak_labels,true_inf_peaks,low_NN,high_NN)

  return statistics_mat/result_trig_iters

def BM1_arrival_times(data_in,end,params):

  cont_deriv_sig = create_SDL_signal(data_in,params[6],1)
  potential_labels = create_SDL_THR_labels(cont_deriv_sig,end,params[0],params[4])
  pot_labels_curr = np.where((potential_labels != 0))[0]
  arrival_times = []
  for idx in pot_labels_curr:
    if np.max(data_in[idx + params[1]:idx + params[2]]) - np.mean(data_in[idx + params[1]:idx + params[2]]) > params[3]:
      arrival_times.append(idx)

  return np.array(arrival_times)

def BM2_arrival_times(data_in,end,params):

  cont_deriv_sig = create_SDL_signal(data_in,params[3],1)
  SDL_labels = create_SDL_THR_labels(cont_deriv_sig,end,params[0],params[1])
  arrival_times = np.where(SDL_labels != 0)[0]

  return arrival_times

def corr_trig_arrival_times(data_in,end,params):

  cont_corr_sig1 = generate_correlation_data_old(data_in,template_n,params[0],params[1])
  cont_corr_sig2 = generate_correlation_data_old(data_in,template_g,params[0],params[1])
  corr_labels = create_Corr_THR_labels(cont_corr_sig1,cont_corr_sig2,end,params[2],params[3])
  arrival_times = np.where(corr_labels != 0)[0]

  return arrival_times

def corr_n_trig_arrival_times(data_in,end,params):

  cont_corr_sig = generate_correlation_data_old(data_in,template_n,params[0],params[1])
  corr_labels = create_SDL_THR_labels(cont_corr_sig,end,params[2],params[3])
  arrival_times = np.where(corr_labels != 0)[0]

  return arrival_times

def corr_g_trig_arrival_times(data_in,end,params):

  cont_corr_sig = generate_correlation_data_old(data_in,template_g,params[0],params[1])
  corr_labels = create_SDL_THR_labels(cont_corr_sig,end,params[2],params[3])
  arrival_times = np.where(corr_labels != 0)[0]

  return arrival_times


def SDL_normal_THR_arrival_times(data_in,end,params):

  cont_deriv_sig = create_SDL_signal(data_in,params[1],params[2])
  SDL_labels = create_SDL_THR_labels(cont_deriv_sig,end,params[0],params[3])
  arrival_times = np.where(SDL_labels != 0)[0]

  return arrival_times

def SDL_find_peaks_arrival_times(data_in,end,params):

  cont_deriv_sig = create_SDL_signal(data_in,params[0],params[1])
  arrival_times = find_peaks(cont_deriv_sig,height = 0.017, distance = params[2], prominence=params[3], width=params[4])

  return arrival_times

def Deep_Trigger_statistics_results(trig_arrival_times,labels,true_inf,curr_low,curr_high):

  arrivals_out, triggers_out, triggered_events_out, false_triggers_array_out = match_pairs(true_inf[:,0],trig_arrival_times,curr_low,curr_high)
  triggered_labels = np.where(false_triggers_array_out == -1, -1, labels[false_triggers_array_out.astype(int)] - 1)
  true_labels = labels[arrivals_out.astype(int)] - 1

  st_idx = max(arrivals_out[1],triggers_out[1])
  en_idx = min(arrivals_out[len(arrivals_out)-1],triggers_out[len(triggers_out)-1])

  st1 = np.argmax(arrivals_out >= st_idx)
  st2 = np.argmax(triggers_out >= st_idx)
  en1 = len(arrivals_out) - np.argmax(arrivals_out[::-1] <= en_idx) - 1
  en2 = len(triggers_out) - np.argmax(triggers_out[::-1] <= en_idx) - 1

  arrivals_out = arrivals_out[st1:en1]
  triggered_events_out = triggered_events_out[st1:en1]
  true_labels = true_labels[st1:en1]
  triggers_out = triggers_out[st2:en2]
  triggered_labels = triggered_labels[st2:en2]

  curr_statistics = calc_deep_trigger_window_statistics(arrivals_out,triggered_events_out,triggered_labels,true_labels)

  return curr_statistics

In [ ]:
def Classifier_Known_Arrivals_All_BM_Results(result_trig_iters,neutrons,gammas):

  statistics_mat = np.zeros((7,10))

  for j in range(result_trig_iters):
    perm = np.random.permutation(np.shape(neutrons)[0])
    neutrons = neutrons[perm]
    gammas = gammas[perm]
    noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                          lambda_g, return_statistics = False, add_noise = True)

    discrete_data, discrete_labels, original_idx_lst,classification_features,attention_features = generate_discrete_data(noised_data,true_inf[:,0],true_inf[:,1],last_arrival_time, extra_features = True, rand_off = False, subtract_baseline = True, WL_classifier = WL_classifier)


    CI_labels, CI_values = charge_integration(discrete_data,CI_parameters)
    corr_labels, corr_values = correlation_classifier(discrete_data,Corr_parameters)
    FC_NN_labels, FC_NN_values = create_labels_Model_FC_NN(model_fc_nn,discrete_data,norm_params_model_fc_nn)
    concat_labels, concat_values = create_labels_Model_Concatenate(model_concat,discrete_data,classification_features,attention_features,norm_params_model_concat)
    attention_labels, attention_values = create_labels_Model_Attention(model_attention,discrete_data,classification_features,attention_features,norm_params_model_attention)

    discrete_data1, discrete_labels1, original_idx_lst1 = generate_discrete_data_BM_NN(noised_data,true_inf[:,0],true_inf[:,1],last_arrival_time, subtract_baseline = True, WL_classifier = WL_classifier)

    BM_NN_acc = test_final_Model_BM_NN(model_bm_nn,discrete_data1,norm_params_model_bm_nn,discrete_labels1)
    BM_NN12_acc = test_final_Model_BM_NN12(discrete_data1,discrete_labels1,model_bm_nn1,norm_params_model_bm_nn1,model_bm_nn2,norm_params_model_bm_nn2)

    statistics_mat[0,:] += Classifiers_statistics_results_all_BM(CI_labels,discrete_labels)
    statistics_mat[1,:] += Classifiers_statistics_results_all_BM(corr_labels,discrete_labels)
    statistics_mat[2,:] += Classifiers_statistics_results_all_BM(FC_NN_labels,discrete_labels)
    statistics_mat[3,:] += Classifiers_statistics_results_all_BM(concat_labels,discrete_labels)
    statistics_mat[4,:] += Classifiers_statistics_results_all_BM(attention_labels,discrete_labels)
    statistics_mat[5,3] += BM_NN_acc
    statistics_mat[6,3] += BM_NN12_acc

  return statistics_mat/result_trig_iters

def Classifiers_statistics_results_all_BM(method_labels,true_labels):

  all_counts = len(true_labels)
  if all_counts == 0:
    acc = 100
  else:
    acc = 100*(np.count_nonzero(method_labels == true_labels)/all_counts)

  num_neutrons = np.count_nonzero(true_labels == 0)
  num_gammas = np.count_nonzero(true_labels == 1)

  num_classified_neutrons = np.count_nonzero(method_labels == 0)
  num_classified_gammas = np.count_nonzero(method_labels == 1)

  error_overall_neutrons = 100*np.abs((num_neutrons - num_classified_neutrons)/num_neutrons)
  error_overall_gammas = 100*np.abs((num_gammas - num_classified_gammas)/num_gammas)

  num_correct_neutrons = np.count_nonzero(np.logical_and(true_labels == 0, true_labels == method_labels))
  num_correct_gammas = np.count_nonzero(np.logical_and(true_labels == 1, true_labels == method_labels))

  if num_neutrons == 0:
    neutron_acc = 100
  else:
    neutron_acc = 100*(num_correct_neutrons/num_neutrons)

  if num_gammas == 0:
    gamma_acc = 100
  else:
    gamma_acc = 100*(num_correct_gammas/num_gammas)

  return all_counts,num_neutrons,num_gammas, acc, neutron_acc, gamma_acc, num_classified_neutrons, num_classified_gammas, error_overall_neutrons, error_overall_gammas

In [ ]:
def Combined_Classifier_statistics_results(method_labels,true_labels):

  num_elements = len(true_labels)
  num_false_alarms = np.count_nonzero(true_labels == -1)
  num_true_detections = num_elements - num_false_alarms

  if num_true_detections == 0:
    acc = 100
  else:
    acc = 100*(np.count_nonzero(method_labels == true_labels)/num_true_detections)

  num_neutrons = np.count_nonzero(true_labels == 0)
  num_gammas = np.count_nonzero(true_labels == 1)


  num_classified_neutrons = np.count_nonzero(method_labels == 0)
  num_classified_gammas = np.count_nonzero(method_labels == 1)

  error_overall_neutrons = 100*np.abs((num_neutrons - num_classified_neutrons)/num_neutrons)
  error_overall_gammas = 100*np.abs((num_gammas - num_classified_gammas)/num_gammas)

  num_correct_neutrons = np.count_nonzero(np.logical_and(true_labels == 0, true_labels == method_labels))
  num_correct_gammas = np.count_nonzero(np.logical_and(true_labels == 1, true_labels == method_labels))

  if num_neutrons == 0:
    neutron_acc = 100
  else:
    neutron_acc = 100*(num_correct_neutrons/num_neutrons)

  if num_gammas == 0:
    gamma_acc = 100
  else:
    gamma_acc = 100*(num_correct_gammas/num_gammas)

  return num_elements,num_neutrons,num_gammas, acc, neutron_acc, gamma_acc, num_classified_neutrons, num_classified_gammas, error_overall_neutrons, error_overall_gammas

In [ ]:
def split_data_for_new_bm(data,labels,N_single,N_double):

  data_n1 = np.zeros((int(2*N_single),WL_classifier))
  data_n2 = np.zeros((int(4*N_double),WL_classifier))
  labels_n1 = np.zeros(int(2*N_single))
  labels_n2 = np.zeros(int(4*N_double))

  j = 0
  k = 0
  for i in range(len(labels)):
    if labels[i] <= 1:
      data_n1[j,:] = data[i,:]
      labels_n1[j] = labels[i]
      j += 1
    else:
      data_n2[k,:] = data[i,:]
      labels_n2[k] = int(labels[i]-2)
      k += 1

  return data_n1, labels_n1, data_n2, labels_n2

def generate_discrete_data_BM_NN(data,arrival_times,arrival_labels,last_arrival_time, subtract_baseline = True, WL_classifier = 200):

  coppied_arrival_times = np.copy(arrival_times)
  coppied_arrival_times = coppied_arrival_times.astype(int)

  N_pulses = 0
  for i in range(len(coppied_arrival_times)):
    if coppied_arrival_times[i] >= cont_starting_time and coppied_arrival_times[i] < last_arrival_time and arrival_labels[i] != -1:
      N_pulses += 1

  discrete_dataset = np.zeros((N_pulses,WL_classifier))
  discrete_labels = np.zeros(N_pulses)
  original_idx_lst = np.zeros(N_pulses)

  j = 0
  i = 0
  while i < len(coppied_arrival_times) - 1:
    if coppied_arrival_times[i] >= cont_starting_time and coppied_arrival_times[i+1] < last_arrival_time and arrival_labels[i] != -1:
      if subtract_baseline == 'True':
        discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier] - data[coppied_arrival_times[i] - time_offset-1]
      else:
        discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier]

      original_idx_lst[j] = i

      if coppied_arrival_times[i+1] - coppied_arrival_times[i] > max_dist:
        discrete_labels[j] = arrival_labels[i]
        i += 1
      else:
        if arrival_labels[i] == 0 and arrival_labels[i+1] == 0:
          discrete_labels[j] = 2
        if arrival_labels[i] == 0 and arrival_labels[i+1] == 1:
          discrete_labels[j] = 3
        if arrival_labels[i] == 1 and arrival_labels[i+1] == 0:
          discrete_labels[j] = 4
        if arrival_labels[i] == 1 and arrival_labels[i+1] == 1:
          discrete_labels[j] = 5
        i += 2
      j += 1
    else:
      i += 1

  return discrete_dataset, discrete_labels, original_idx_lst


def create_BM_FC_data_for_training(neutrons,gammas,N_single,N_double):

  discrete_data_out = np.zeros((int(2*N_single + 4*N_double),WL_classifier))
  discrete_labels_out = np.zeros(int(2*N_single + 4*N_double))
  j = 0
  count_lst = np.zeros(6)
  count_thr_lst = np.array([N_single,N_single,N_double,N_double,N_double,N_double])

  while(True):
    perm = np.random.permutation(np.shape(neutrons)[0])
    neutrons = neutrons[perm]
    gammas = gammas[perm]
    noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                          lambda_g, return_statistics = False, add_noise = True)

    discrete_data, discrete_labels, original_idx_lst = generate_discrete_data_BM_NN(noised_data,true_inf[:,0],true_inf[:,1],last_arrival_time, subtract_baseline = True, WL_classifier = WL_classifier)

    for i in range(len(discrete_labels)):
      if count_lst[int(discrete_labels[i])] < count_thr_lst[int(discrete_labels[i])]:
        discrete_data_out[j,:] = discrete_data[i,:]
        discrete_labels_out[j] = discrete_labels[i]
        count_lst[int(discrete_labels[i])] += 1
        j += 1
      if j == int(2*N_single + 4*N_double):
        perm = np.random.permutation(np.shape(discrete_data_out)[0])
        discrete_data_out = discrete_data_out[perm]
        discrete_labels_out = discrete_labels_out[perm]

        return discrete_data_out, discrete_labels_out


In [ ]:
def test_final_Model_BM_NN12(data,true_labels,model1,params1,model2,params2):

  method_labels = np.zeros(len(true_labels))

  for i in range(len(true_labels)):
    if true_labels[i] <= 1:
      data1 = (data[i,:] - params1[0])/params1[1]
      data1 = torch.Tensor(data1)
      nn_values = model1(data1)
      method_labels[i] = torch.round(nn_values)
    else:
      data2 = (data[i,:] - params2[0])/params2[1]
      data2 = torch.Tensor(data2)
      nn_values = model2(data2)
      method_labels[i] = torch.argmax(nn_values) + 2

  allc = 0
  corrc = 0

  for i in range(len(true_labels)):
    if true_labels[i] <= 1:
      allc += 1
    else:
      allc += 2
    if true_labels[i] == method_labels[i]:
      if true_labels[i] <= 1:
        corrc += 1
      else:
        corrc += 2
    if true_labels[i] == 2 and (method_labels[i] == 3 or method_labels[i] == 4):
      corrc += 1
    if true_labels[i] == 5 and (method_labels[i] == 3 or method_labels[i] == 4):
      corrc += 1
    if true_labels[i] == 3 and (method_labels[i] == 2 or method_labels[i] == 5):
      corrc += 1
    if true_labels[i] == 4 and (method_labels[i] == 2 or method_labels[i] == 5):
      corrc += 1

  if allc == 0:
    acc = 100
  else:
    acc = 100*(corrc/allc)

  return acc

def create_labels_Model_BM_NN(model,data,params):

  data = (data - params[0])/params[1]
  data = torch.Tensor(data)
  nn_values = model(data)
  labels_out = torch.argmax(nn_values,axis=1)

  return np.squeeze(labels_out.detach().numpy()), np.squeeze(nn_values.detach().numpy())

def test_Model_BM_NN(data,arrival_times,arrival_labels,last_arrival_time, subtract_baseline = True, WL_classifier = 200):

  coppied_arrival_times = np.copy(arrival_times)
  coppied_arrival_times = coppied_arrival_times.astype(int)

  N_pulses = 0
  for i in range(len(coppied_arrival_times)):
    if coppied_arrival_times[i] >= cont_starting_time and coppied_arrival_times[i] < last_arrival_time and arrival_labels[i] != -1:
      N_pulses += 1

  discrete_dataset = np.zeros((N_pulses,WL_classifier))
  discrete_labels = np.zeros(N_pulses)
  original_idx_lst = np.zeros(N_pulses)

  j = 0
  i = 0
  while i < len(coppied_arrival_times) - 1:
    if coppied_arrival_times[i] >= cont_starting_time and coppied_arrival_times[i+1] < last_arrival_time and arrival_labels[i] != -1:
      if subtract_baseline == 'True':
        discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier] - data[coppied_arrival_times[i] - time_offset-1]
      else:
        discrete_dataset[j,:] = data[coppied_arrival_times[i] - time_offset:coppied_arrival_times[i] - time_offset + WL_classifier]

      original_idx_lst[j] = i

      if coppied_arrival_times[i+1] - coppied_arrival_times[i] > max_dist:
        discrete_labels[j] = arrival_labels[i]
        i += 1
      else:
        if arrival_labels[i] == 0 and arrival_labels[i+1] == 0:
          discrete_labels[j] = 2
        if arrival_labels[i] == 0 and arrival_labels[i+1] == 1:
          discrete_labels[j] = 3
        if arrival_labels[i] == 1 and arrival_labels[i+1] == 0:
          discrete_labels[j] = 4
        if arrival_labels[i] == 1 and arrival_labels[i+1] == 1:
          discrete_labels[j] = 5
        i += 2
      j += 1
    else:
      i += 1

  return discrete_dataset, discrete_labels, original_idx_lst

def test_final_Model_BM_NN(model,data,params,true_labels):

  data = (data - params[0])/params[1]
  data = torch.Tensor(data)
  nn_values = model(data)
  method_labels = np.zeros(len(true_labels))

  for i in range(len(true_labels)):
    if true_labels[i] <= 1:
      method_labels[i] = torch.argmax(nn_values[i,:2])
    else:
      method_labels[i] = torch.argmax(nn_values[i,2:]) + 2

  allc = 0
  corrc = 0

  for i in range(len(true_labels)):
    if true_labels[i] <= 1:
      allc += 1
    else:
      allc += 2
    if true_labels[i] == method_labels[i]:
      if true_labels[i] <= 1:
        corrc += 1
      else:
        corrc += 2
    if true_labels[i] == 2 and (method_labels[i] == 3 or method_labels[i] == 4):
      corrc += 1
    if true_labels[i] == 5 and (method_labels[i] == 3 or method_labels[i] == 4):
      corrc += 1
    if true_labels[i] == 3 and (method_labels[i] == 2 or method_labels[i] == 5):
      corrc += 1
    if true_labels[i] == 4 and (method_labels[i] == 2 or method_labels[i] == 5):
      corrc += 1

  if allc == 0:
    acc = 100
  else:
    acc = 100*(corrc/allc)

  return acc

def Classifiers_statistics_results_BM_NN1(method_labels,true_labels):

  allc = 0
  corrc = 0

  for i in range(len(true_labels)):
    if true_labels[i] == 0:
      p1t = 0
      p2t = -1
    if true_labels[i] == 1:
      p1t = 1
      p2t = -1
    if true_labels[i] == 2:
      p1t = 0
      p2t = 0
    if true_labels[i] == 3:
      p1t = 1
      p2t = 0
    if true_labels[i] == 4:
      p1t = 0
      p2t = 1
    if true_labels[i] == 5:
      p1t = 1
      p2t = 1

    if method_labels[i] == 0:
      p1p = 0
      p2p = -1
    if method_labels[i] == 1:
      p1p = 1
      p2p = -1
    if method_labels[i] == 2:
      p1p = 0
      p2p = 0
    if method_labels[i] == 3:
      p1p = 1
      p2p = 0
    if method_labels[i] == 4:
      p1p = 0
      p2p = 1
    if method_labels[i] == 5:
      p1p = 1
      p2p = 1


    if p2t == -1 and p2p == -1:
      if p1t == p1p:
        corrc += 1
      allc += 1
    if p2t != -1 and p2p != -1:
      if p1t == p1p:
        corrc += 1
      if p2t == p2p:
        corrc += 1
      allc += 2
    if p2t != -1 and p2p == -1:
      if p1p == p1t or p1p == p2t:
        corrc += 1
      allc += 2
    if p2t == -1 and p2p != -1:
      if p1p == p1t or p2p == p1t:
        corrc += 1
      allc += 1

  if allc == 0:
    acc = 100
  else:
    acc = 100*(corrc/allc)

  return acc

def Classifiers_statistics_results_BM_NN2(method_labels,true_labels):

  allc = 0
  corrc = 0

  for i in range(len(true_labels)):
    if true_labels[i] == 0:
      p1t = 0
      p2t = -1
    if true_labels[i] == 1:
      p1t = 1
      p2t = -1
    if true_labels[i] == 2:
      p1t = 0
      p2t = 0
    if true_labels[i] == 3:
      p1t = 1
      p2t = 0
    if true_labels[i] == 4:
      p1t = 0
      p2t = 1
    if true_labels[i] == 5:
      p1t = 1
      p2t = 1

    if method_labels[i] == 0:
      p1p = 0
      p2p = -1
    if method_labels[i] == 1:
      p1p = 1
      p2p = -1
    if method_labels[i] == 2:
      p1p = 0
      p2p = 0
    if method_labels[i] == 3:
      p1p = 1
      p2p = 0
    if method_labels[i] == 4:
      p1p = 0
      p2p = 1
    if method_labels[i] == 5:
      p1p = 1
      p2p = 1


    if p2t == -1 and p2p == -1:
      if p1t == p1p:
        corrc += 1
      allc += 1
    if p2t != -1 and p2p != -1:
      if p1t == p1p:
        corrc += 1
      if p2t == p2p:
        corrc += 1
      allc += 2
    if p2t != -1 and p2p == -1:
      if p1p == p1t or p1p == p2t:
        corrc += 1
      allc += 2
    if p2t == -1 and p2p != -1:
      if p1p == p1t or p2p == p1t:
        corrc += 1
      allc += 2

  if allc == 0:
    acc = 100
  else:
    acc = 100*(corrc/allc)

  return acc


In [ ]:
def Combined_Results_Function(result_trig_iters,neutrons,gammas):

  statistics_mat = np.zeros(18)

  for j in range(result_trig_iters):
    perm = np.random.permutation(np.shape(neutrons)[0])
    neutrons = neutrons[perm]
    gammas = gammas[perm]
    noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons,gammas,lambda_n,
                                                                                          lambda_g, return_statistics = False, add_noise = True)

    deep_trig_arrival_times = deep_trigger_arrival_times(noised_data, last_arrival_time, prob=True)

    arrivals_out, triggers_out, triggered_events_out, false_triggers_array_out = match_pairs(true_inf[:,0],deep_trig_arrival_times,low_NN,high_NN)

    triggered_labels = np.where(false_triggers_array_out == -1, -1, labels[false_triggers_array_out.astype(int)] - 1)
    true_labels = labels[arrivals_out.astype(int)] - 1

    st_idx = max(arrivals_out[1],triggers_out[1])
    en_idx = min(arrivals_out[len(arrivals_out)-1],triggers_out[len(triggers_out)-1])

    st1 = np.argmax(arrivals_out >= st_idx)
    st2 = np.argmax(triggers_out >= st_idx)
    en1 = len(arrivals_out) - np.argmax(arrivals_out[::-1] <= en_idx) - 1
    en2 = len(triggers_out) - np.argmax(triggers_out[::-1] <= en_idx) - 1

    arrivals_out = arrivals_out[st1:en1]
    triggered_events_out = triggered_events_out[st1:en1]
    true_labels = true_labels[st1:en1]
    triggers_out = triggers_out[st2:en2]
    triggered_labels = triggered_labels[st2:en2]

    discrete_data, discrete_labels, original_idx_lst,classification_features,attention_features = generate_discrete_data_with_FA(noised_data,triggers_out,triggered_labels,last_arrival_time, extra_features = True, rand_off = False, subtract_baseline = True, WL_classifier = WL_classifier)

    attention_labels, attention_values = create_labels_Model_Attention(model_attention,discrete_data,classification_features,attention_features,norm_params_model_attention)

    statistics_mat[:8] += calc_deep_trigger_window_statistics(arrivals_out,triggered_events_out,triggered_labels,true_labels)
    statistics_mat[8:] += Combined_Classifier_statistics_results(attention_labels,discrete_labels)

  return statistics_mat/result_trig_iters

In [ ]:
def divide_dist_to_n_g(method_values,original_labels):

  method_values_n = []
  method_values_g = []
  for i in range(len(original_labels)):
    if original_labels[i] == 0:
      method_values_n.append(method_values[i])
    if original_labels[i] == 1:
      method_values_g.append(method_values[i])

  return method_values_n, method_values_g